In [13]:
import numpy as np
import pandas as pd
import os
import csv
import statistics

from synergy_dataset import Dataset, iter_datasets
from sentence_transformers import SentenceTransformer, util
import torch

In [57]:
def cria_dataset(dataset):
    ds = Dataset(dataset)
    ds = ds.to_frame()
    ds = ds.fillna('')
    title = ds['title']
    abstract = ds['abstract']
    X = np.array([x[0] + ' ' + x[1] for x in zip(title, abstract)])
    y = np.array(ds['label_included'])

    return X, y

def torch_delete(tensor, indices):
    mask = torch.ones(tensor.shape[0], dtype=torch.bool)
    mask[indices] = False
    return tensor[mask]

In [53]:
# model_checkpoint = 'all-distilroberta-v1'
# model_checkpoint = 'all-mpnet-base-v2'
model_checkpoint = 'all-MiniLM-L6-v2'
dataset = 'Muthu_2021'

X, y = cria_dataset(dataset)
model = SentenceTransformer(model_checkpoint)
X_emb = model.encode(X, convert_to_tensor=True)

/home/junior/Mestrado Comp Ifes/Dissertação/cosine-similarity/.venv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


###Selecionando o item que tiver a maior similaridade a cada iteração

In [58]:
n_iter = 50
n_items_inicial = 2
# np.random.seed(42)

labels_1_idx = [i for i, _ in enumerate(y) if y[i] == 1]
indices_escolhidos = np.random.choice(labels_1_idx, n_items_inicial, replace=False)
abstract_emb = X_emb[indices_escolhidos]
print(X[indices_escolhidos])
print('\n')

X_pool = torch_delete(X_emb, indices_escolhidos)
# X_pool = np.delete(X_emb, indices_escolhidos, axis=0)
y_pool = np.delete(y, indices_escolhidos)

quant = 0

for iter in range(n_iter):
    results=[]
    # print(abstract_emb.shape)
    for index, (label, abstract) in enumerate(zip(y_pool, X_pool)):
        similarity = util.cos_sim(abstract_emb, abstract)
        media = sum([item[0] for item in similarity.tolist()])/len(similarity.tolist())
        results.append((index, label, media, abstract))
    results.sort(key=lambda x: x[2], reverse=True)
    for result in results[:5]:
        print(result[:3])
    print('\n')
    # abstract_emb = torch.cat((abstract_emb, results[0][3].reshape(1, 768)), 0)
    abstract_emb = torch.cat((abstract_emb, results[0][3].reshape(1, 384)), 0)
    # abstract_emb = np.append(abstract_emb, results[0][3].reshape(1, 768), axis=0)
    # abstract_emb = np.append(abstract_emb, results[0][3].reshape(1, 384), axis=0)
    # X_pool = torch.cat([X_pool[0:results[0][0]], X_pool[results[0][0] + 1:]])
    X_pool = torch_delete(X_pool, results[0][0])
    # X_pool = np.delete(X_pool, results[0][0], axis=0)
    y_pool = np.delete(y_pool, results[0][0])
    if results[0][1] == 1:
        quant += 1
print(f'Total 1: {quant}')
print(f'Percentual 1: {quant/n_iter}')



['Cement Directed Kyphoplasty Reduces Cement Leakage as Compared With Vertebroplasty A novel randomized, controlled, unblinded clinical trial comparing 2 procedural interventions for painful osteoporotic vertebral compression fractures.The primary study objective was to evaluate cement leakage for a cement directed kyphoplasty system (CDKS) with anteriorly biased cement flow and vertebroplasty. The secondary study objective was to compare adjacent level fracture rates and vertebral body height for these 2 intervention methods.Cement leakage remains a significant clinical problem associated with vertebroplasty and kyphoplasty procedures. Uncontrolled cement flow in the posterior direction can result in leakage into the vertebral veins or spinal canal, leading to potentially serious clinical complications.Seventy-seven patients with painful osteoporotic vertebral compression fractures were enrolled. Patients were randomized 2:1 for treatment with CDKS (49 patients, 65 levels) or vertebro

###Selecionando 3 dos items que tiverem a maior similaridade a cada iteração

In [9]:
n_iter = 50
n_items_inicial = 4
# np.random.seed(42)

X_emb = model.encode(X)

labels_1_idx = [i for i, _ in enumerate(y) if y[i] == 1]
indices_escolhidos = np.random.choice(labels_1_idx, n_items_inicial, replace=False)
abstract_emb = X_emb[indices_escolhidos]
X_pool = np.delete(X_emb, indices_escolhidos, axis=0)
y_pool = np.delete(y, indices_escolhidos)

quant = 0

for iter in range(n_iter):
    results=[]
    for index, (label, abstract) in enumerate(zip(y_pool, X_pool)):
        similarity = util.cos_sim(abstract_emb, abstract)
        media = sum([item[0] for item in similarity.tolist()])/len(similarity.tolist())
        results.append((index, label, media, abstract))
    results.sort(key=lambda x: x[2], reverse=True)
    for result in results[:5]:
        print(result[:3])
    print('\n')
    for result in results[:3]:
        abstract_emb = np.append(abstract_emb, result[3].reshape(1, 768), axis=0)
        X_pool = np.delete(X_pool, result[0], axis=0)
        y_pool = np.delete(y_pool, result[0])



(2416, 0, 0.7168474197387695)
(945, 1, 0.7150405645370483)
(1291, 0, 0.71299809217453)
(1388, 1, 0.7126059681177139)
(1150, 1, 0.7050378769636154)


(1290, 0, 0.7861907993044172)
(554, 0, 0.7807255387306213)
(1066, 0, 0.7781424522399902)
(1386, 1, 0.7689688461167472)
(2423, 0, 0.767947827066694)


(1065, 0, 0.8175256431102753)
(2420, 0, 0.7932308375835418)
(1383, 1, 0.7912881910800934)
(1272, 1, 0.7890381932258606)
(1456, 0, 0.7847175419330596)


(2418, 0, 0.8196079501738915)
(1382, 1, 0.8184734903849088)
(1271, 1, 0.7991819427563593)
(1454, 0, 0.797363267495082)
(505, 0, 0.7953663651759808)


(1452, 0, 0.8050455152988434)
(505, 0, 0.8038392327725887)
(193, 0, 0.7955972924828529)
(1915, 0, 0.7901510111987591)
(2608, 0, 0.7877300642430782)


(1912, 0, 0.7967291286117152)
(1065, 0, 0.7910697554287157)
(2605, 0, 0.788271897717526)
(192, 0, 0.7877835537257948)
(668, 0, 0.7874812326933208)


(2603, 0, 0.792538271708922)
(668, 0, 0.791521048003977)
(192, 0, 0.7885371392423456)
(829, 0, 0.786